# 사전 환경 만들기

## 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 필요한 라이브러리 설치 및 임포트

In [ ]:
!pip install evaluate # Install the 'evaluate' package
from datasets import load_from_disk, Dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import numpy as np
from evaluate import load as load_metric # Import after installing the package

## GPU 사용 가능 여부 확인

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# 데이터 준비

## 데이터셋 경로 설정 (경로는 사용자마다 다름)

In [ ]:
# 전체 데이터셋 로드
train_dataset = load_from_disk("/content/drive/MyDrive/Colab Notebooks/wav2vec classification/train_dataset")
test_dataset = load_from_disk("/content/drive/MyDrive/Colab Notebooks/wav2vec classification/test_dataset")

# train_dataset 40,000개
# test_dataset 10,000개
# 8대2 (4대1)
# 1,000개 -> 800:200


# total_data_number = 10000
# train_data_number = total_data_number // 5 * 4
# test_data_number = total_data_number // 5

# train_dataset = train_dataset.select(range(train_data_number))
# test_dataset = test_dataset.select(range(test_data_number))


## 학습용 데이터와 검증용 데이터로 나누기

## Wav2Vec2 모델과 프로세서 불러오기

In [ ]:
model_name = "facebook/wav2vec2-base"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)


# 모델 만들고 학습하기

## 2. Low-Rank Approximation

In [ ]:
def apply_low_rank_approximation(model, rank=1):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            # 저랭크 근사 적용
            weight = module.weight.data.cpu().numpy()
            U, S, Vt = np.linalg.svd(weight, full_matrices=False)
            U = U[:, :rank]
            S = np.diag(S[:rank])
            Vt = Vt[:rank, :]
            new_weight = torch.tensor(U @ S @ Vt, dtype=module.weight.dtype)
            module.weight.data = new_weight.to(module.weight.device)

apply_low_rank_approximation(model)


## 모델 학습을 위한 파라미터 설정

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    report_to="none",
)

## 학습 루프 구현

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return metric.compute(predictions=preds, references=labels)

# DataCollatorWithPadding을 사용하여 데이터의 길이를 맞춤
data_collator = DataCollatorWithPadding(tokenizer=processor, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=processor,
    compute_metrics=compute_metrics
)


## 모델 학습

In [ ]:
# 시간 측정을 하기 위해  %%time을 추가했음

%%time

trainer.train()

# 모델 평가

In [ ]:
# 평가
# 위의 trainer.train() 코드에서 이미 validation accuracy를 알려주기 때문에 이 코드를 돌릴 필요는 없음
# results = trainer.evaluate()
# print(results)